# Creating/Using Image Classifier


### Step 1 - Create Data for Classifier

In [2]:
import cv2

### Load HAAR Classifier


In [ ]:
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def face_extractor(image):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_image, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = image[y:y+h, x:x+w]

    return cropped_face

In [ ]:
video = cv2.VideoCapture(0)
count = 0

while(True):
    ret, frame = video.read()
    if face_extractor(frame) is not None:
        count = count + 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
video.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Step 2 - Train Model

In [ ]:
import numpy as np
from os import listdir
from os.path import isfile, join

In [ ]:
# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [ ]:
# Create arrays for training data and labels
training_data, labels = [], []

In [ ]:
# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    training_data.append(np.asarray(images, dtype=np.uint8))
    labels.append(i)

In [ ]:
# Create a numpy array for both training data and labels
labels = np.asarray(labels, dtype=np.int32)

In [ ]:
# Initialize facial recognizer
model = cv2.face.LBPHFaceRecognizer_create()

In [ ]:
# Let's train our model 
model.train(np.asarray(training_data), np.asarray(labels))
print("Model trained sucessefully")

### Step 3 - Save the model to external file

In [ ]:
model.save('custom-classifier.xml')

### Step 3 - Run Our Facial Recognition

In [ ]:
## loading our custom created model to classify images
new_model = cv2.face.LBPHFaceRecognizer_create()
new_model.read('custom-classifier.xml')

In [ ]:
def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_image, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

In [ ]:
# Open Webcam
video = cv2.VideoCapture(0)

while True:

    ret, frame = video.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        results = new_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) & 0xff==ord('q'): #13 is the Enter Key
        break
        
video.release()
cv2.destroyAllWindows()     